In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def paraphrase(sentence, model, tokenizer):
    prompt =f"Answer the following following english paraphrase question. what is the paraphrase of the following sentence if you cannot repeat the verb phrases used in the given sentence? {sentence}"
    messages = [
        {"role": "system", "content": "You are Qwen, you are tasked to paraphrase english sentences that are given to you."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

sentence = "omg this movie is so good"
paraphrase(sentence, model, tokenizer)

# Summarization model below

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

ARTICLE = """Answer the following following english paraphrase question. what is the paraphrase of the following sentence if you cannot repeat the verb phrases used in the given sentence? what the heck the movie is so bad, hated the actors
"""
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))
